# Part 2, Topic 1, Lab A: Instruction Power Differences (HARDWARE)

---
**THIS IS NOT THE COMPLETE TUTORIAL - see file with `(MAIN)` in the name.**

---

First you'll need to select which hardware setup you have. You'll need to select a `SCOPETYPE`, a `PLATFORM`, and a `CRYPTO_TARGET`. `SCOPETYPE` can either be `'OPENADC'` for the CWLite/CW1200 or `'CWNANO'` for the CWNano. `PLATFORM` is the target device, with `'CWLITEARM'`/`'CW308_STM32F3'` being the best supported option, followed by `'CWLITEXMEGA'`/`'CW308_XMEGA'`, then by `'CWNANO'`. For example:

```python
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
```

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEXMEGA'

## Creating a New Directory

The following bash block will create a new directory and copy the `simpleserial-base` code into it:

In [2]:
%%bash
cd ../../../hardware/victims/firmware/
mkdir -p simpleserial-base-lab2 && cp -r simpleserial-base/* $_
cd simpleserial-base-lab2

## Building Firmware

Rerun this every time you make changes to your code:

In [3]:
%%bash -s "$PLATFORM"
cd ../../../hardware/victims/firmware/simpleserial-base-lab2
make PLATFORM=$1 CRYPTO_TARGET=NONE

SS_VER set to SS_VER_1_1
rm -f -- simpleserial-base-CWLITEXMEGA.hex
rm -f -- simpleserial-base-CWLITEXMEGA.eep
rm -f -- simpleserial-base-CWLITEXMEGA.cof
rm -f -- simpleserial-base-CWLITEXMEGA.elf
rm -f -- simpleserial-base-CWLITEXMEGA.map
rm -f -- simpleserial-base-CWLITEXMEGA.sym
rm -f -- simpleserial-base-CWLITEXMEGA.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-base.s simpleserial.s XMEGA_AES_driver.s uart.s usart_driver.s xmega_hal.s
rm -f -- simpleserial-base.d simpleserial.d XMEGA_AES_driver.d uart.d usart_driver.d xmega_hal.d
rm -f -- simpleserial-base.i simpleserial.i XMEGA_AES_driver.i uart.i usart_driver.i xmega_hal.i
mkdir .dep
.
Welcome to another exciting ChipWhisperer target build!!
avr-gcc (GCC) 10.1.0
Copyright (C) 2020 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling C: simpleserial-base.c
avr-gcc -c -m

.././simpleserial/simpleserial.c:18: warning: "CRC" redefined
   18 | #define CRC 0xA6
      | 
In file included from c:\users\gc_lu\chipwh~1\cw\home\portable\avrgcc\avr-gcc-10.1.0-x64-windows\avr\include\avr\io.h:552,
                 from .././hal/hal.h:93,
                 from .././simpleserial/simpleserial.c:5:
c:\users\gc_lu\chipwh~1\cw\home\portable\avrgcc\avr-gcc-10.1.0-x64-windows\avr\include\avr\iox128d3.h:2239: note: this is the location of the previous definition
 2239 | #define CRC    (*(CRC_t *) 0x00D0)  /* Cyclic Redundancy Checker */
      | 


## Connecting to Hardware

Let's setup the ChipWhisperer

In [4]:
import chipwhisperer as cw
try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()
   
try:
    target = cw.target(scope)
except IOError:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope)

print("INFO: Found ChipWhisperer😍")

if "STM" in PLATFORM or PLATFORM == "CWLITEARM" or PLATFORM == "CWNANO":
    prog = cw.programmers.STM32FProgrammer
elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
    prog = cw.programmers.XMEGAProgrammer
else:
    prog = None
    
import time
time.sleep(0.05)
scope.default_setup()
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.05)
        scope.io.pdic = 'high_z' #XMEGA doesn't like pdic driven high
        time.sleep(0.05)
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high_z'
        time.sleep(0.05)

INFO: Found ChipWhisperer😍


## Uploading Firmware

The following code will upload firmware to the target. Run it after you've built the firmware:

In [5]:
cw.program_target(scope, prog, "../../../hardware/victims/firmware/simpleserial-base-lab2/simpleserial-base-{}.hex".format(PLATFORM))

XMEGA Programming flash...
XMEGA Reading flash...
Verified flash OK, 2097 bytes


## Capturing a Power Trace

The following is a short function to capture a power trace. We're using `cw.capture_trace()`, which is a short function to replace what we did last time. Check our `readthedocs` page for API documentation: https://chipwhisperer.readthedocs.io/

In [6]:
def capture_trace(_ignored=None):
    ktp = cw.ktp.Basic()
    key, text = ktp.next()
    return cw.capture_trace(scope, target, text).wave